In [ ]:
import pandas as pd
import numpy as np
import requests
import re
import osmnx as ox
import geopandas as gpd

df = pd.read_csv('your csv file')
df

In [ ]:
filtered_df = df[df['polygon'].isnull() | (df['polygon'] == '')]

# Extract mall_name from the filtered DataFrame
emp_df = filtered_df['mall_name']

emp_df

In [ ]:
mall_names = []
for i in emp_df:
    mall_names.append(i)

mall_names

In [ ]:
# Function to format polygon points as a string in the required format
def format_polygon_points(polygon_points):
    formatted_points = ", ".join([f"{lon} {lat}" for lat, lon in polygon_points])
    return f"POLYGON (({formatted_points}))"
    
# Function to get polygon points from OpenStreetMap within the Philippines boundary
def get_polygon_points(place_name):
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    // Define the boundary for the Philippines
    area[name="Philippines"];
    // Query for the place name within the Philippines boundary
    area[name="{place_name}"](area);
    // Output the geometry of the area
    out geom;
    """
    response = requests.post(overpass_url, data=overpass_query)
    data = response.json()
    if 'elements' in data and len(data['elements']) > 0:
        element = data['elements'][0]
        if 'type' in element and element['type'] == 'way':
            # Extract polygon points
            polygon_points = [(node['lat'], node['lon']) for node in element['geometry']]
            return format_polygon_points(polygon_points)
    return None

In [ ]:
# Alternative functions to get polygons
def format_polygon_points2(polygon_points):
    formatted_points = ", ".join([f"{lat} {lon}" for lat, lon in polygon_points])
    return f"POLYGON (({formatted_points}))"

def get_polygon_points2(mall_name):
    try:
        area = ox.geocode_to_gdf(mall_name)
        if area.empty:
            print(f"Mall Name: {mall_name} Polygon_points: not found")
            return []
        polygon_points = []
        for geometry in area.geometry:
            polygon_points.extend(geometry.exterior.coords)
        return format_polygon_points2(polygon_points)
    except (ox.geocoder.InsufficientResponseError, TypeError, AttributeError):
        print(f"Mall Name: {mall_name} Polygon_points: Polygon points not found again")
        return []

In [ ]:
remaining = [] # Paste remaining values from mall_name list here

new = [] # list for Data Frame

for mall in remaining:
    poly = get_polygon_points2(mall)
    if poly:
        new.append([mall, poly])
        print(f"Mall Name: {mall}")
        print()
        print(get_polygon_points2(mall))
        print()
        print()
    else:
        poly = None
        new.append([mall, poly])
        print(f"Mall Name: {mall} Polygon_points: not found")

new_df = pd.DataFrame(new, columns=['mall_name', 'polygon'])


In [ ]:
new_df 

In [ ]:
new_df.to_csv('left-over-manila.csv', sep=',', index=False, encoding='utf-8')